In [3]:
binary_int = int("11000010110001001100011", 2);
binary_array = binary_int.to_bytes(3, "big")
ascii_text = binary_array.decode()
print(ascii_text)

abc


In [37]:
import numpy as np

In [38]:
def init_populations(chrom_length,popu_length):

    individuals = []
    for i in range(popu_length):
        chromosome = []
        for j in range(chrom_length):
            p = np.random.randint(32,126)
            chromosome.append(p)
        individuals.append(chromosome)
    return individuals

In [59]:
ch1=init_populations(32,10)
ch1[0]

[40,
 112,
 104,
 73,
 100,
 50,
 79,
 59,
 110,
 96,
 90,
 66,
 93,
 99,
 37,
 71,
 54,
 109,
 109,
 67,
 49,
 101,
 82,
 61,
 43,
 48,
 111,
 59,
 94,
 85,
 48,
 97]

In [75]:
def decode(chromosome):
    decoded=[]
    for letter in chromosome:
        binary_array = letter.to_bytes(1, "big")
        ascii_text = binary_array.decode()
        decoded.append(ascii_text)
    decoded="".join(decoded)
    return decoded

In [74]:
decode(ch1[0])

<class 'str'>


'(phId2O;n`ZB]c%G6mmC1eR=+0o;^U0a'

In [54]:
def fitness(chromosome,target):
    decoded_chromosome=decode(chromosome)
    count=0
    for i in range(len(target)):
        if decoded_chromosome[i]==target[i]:
            count+=1
    
    return count

In [102]:
target="Me llamo Juan y mi compa Manuel."
fitness(ch1[0],target)

0

In [100]:
def prob_fitness(pop,target):
    probs = []
    cumulative_prob = []
    fitnesses = []
    acc = 0

    for i in range(len(pop)):
        fitnesses.append( fitness(pop[i],target) )
        acc += fitnesses[i]
        cumulative_prob.append(acc)

    for j in range(len(pop)):
        if acc!=0:
            probs.append(fitnesses[j]/acc)
        else:
            probs.append(0)

    return probs, cumulative_prob,fitnesses

In [103]:
prob_fitness(ch1,target)

([0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0],
 [0, 0, 0, 0, 1, 1, 1, 2, 2, 2],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0])

In [61]:
def roullete(cumulative_fit,pop):
    selected_ind = []

    for i in range(2):
        ran = np.random.rand() * cumulative_fit[-1]

        s = 0

        while ran > cumulative_fit[s]:
            s += 1

        selected_ind.append(pop[s])
    return selected_ind

In [81]:
def crossover(p_cross, cumulative_fit, popu_length,pop,chrom_length):
    
    new_gen = []

    while len(new_gen)<popu_length:
        
        selected_ind = roullete(cumulative_fit,pop)

        ind1 = selected_ind[0]
        ind2 = selected_ind[1]
        # print("ind1",decode(ind1))
        # print("ind2",decode(ind2))
        p = np.random.rand()

        if p < p_cross:

            cut = np.random.randint(1,chrom_length-1)
            # print(cut)
            new_ind1 = ind1[:cut] + ind2[cut:]
            new_ind2 = ind2[:cut] + ind1[cut:]
            # print("new_ind1",decode(new_ind1))
            # print("new_ind2",decode(new_ind2))
            new_gen.append(new_ind1)
            new_gen.append(new_ind2)
            # print("\n")
    return new_gen

In [82]:
p_cross , cumulative_fit , popu_length , chrom_length = 0.9 , prob_fitness(ch1,target)[1] , 10 , 32
new_gen=crossover(p_cross, cumulative_fit, popu_length,ch1,chrom_length)

In [91]:
def mutation(new_gen,p_mutation,chrom_length):

    for i in range(len(new_gen)):
        # print("before",new_gen[i])
        for j in range(chrom_length):
            p= np.random.rand()
            if p<p_mutation:
                letter=np.random.randint(32,126)
                if new_gen[i][j]!=letter:
                    new_gen[i][j]=letter
                else:
                    while new_gen[i][j]!=letter:
                        letter=np.random.randint(32,126)
                        new_gen[i][j]=letter
        # print("after",new_gen[i])
    return new_gen

In [92]:
new_mutated=mutation(new_gen,0.04,chrom_length)

In [107]:
def genetic_algo(target,p_cross,p_mutation,popu_length,chrom_length,number_gen,random_state):
    np.random.seed(random_state)
    
    min, max, mean, best_chrom = [], [],[], []

    pop = init_populations(chrom_length,popu_length)
    probs, cumulative_fit, fitnesses = prob_fitness(pop,target)
    best_chrom.append(pop[np.argmax(fitnesses)])
    
    min.append(np.min(fitnesses))
    max.append(np.max(fitnesses))
    mean.append(np.mean(fitnesses))
    
    for i in range(number_gen):
        new_gen = crossover(p_cross, cumulative_fit, popu_length,pop,chrom_length)
        new_gen = mutation(new_gen,p_mutation,chrom_length)
        probs, cumulative_fit, fitnesses = prob_fitness(new_gen,target)
   
        best_chrom.append(new_gen[np.argmax(fitnesses)])
        if (i+1)%100==0:
            print(f"Generation {i+1}: {decode(best_chrom[-1])}")
        min.append(np.min(fitnesses))
        max.append(np.max(fitnesses))
        mean.append(np.mean(fitnesses))

    return min, max, mean, best_chrom

In [108]:
p_cross=0.9
popu_length=100
p_mutation=0.01
number_gen=1000
rand_state=42
target="hola"
chrom_length=len(target)

min, max, mean, best_chrom = genetic_algo(target,p_cross,p_mutation,popu_length,chrom_length,number_gen,rand_state)
best_chrom

Generation 100: 26-O
Generation 200: 26-O
Generation 300: 26-O
Generation 400: 26-O
Generation 500: 26-O
Generation 600: 26-O
Generation 700: 26-O
Generation 800: 26-O
Generation 900: 26-O
Generation 1000: 26-O


[[75, 83, 108, 39],
 [104, 61, 116, 97],
 [40, 61, 116, 97],
 [85, 77, 39, 78],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 98, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 45, 79],
 [50, 54, 108, 79],
 [60, 114, 114, 60],
 [50,